# Task 1

Christoph's task:

* Load 2FHL data
* Make a source counts image - fill events into a sky image, show the "cluster" of sources
* Make an energy distribution curve

## Setup

In [37]:
# import warnings
# warnings.filterwarnings("ignore", category=UserWarning, module='astropy')

%matplotlib inline
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
# plt.style.use('ggplot')

from astropy.table import Table
from gammapy.image import SkyImage
from gammapy.catalog import source_catalogs

## Load data

I'm loading the 2FHL event list.

In [43]:
table = Table.read('../datasets/2fhl/2fhl_events.fits.gz')

# table[:5].show_in_browser(jsviewer=True) # <-- show table in browser tab
table[:5]                                  # <-- show table in notebook

ENERGY,RA,DEC,L,B,THETA,PHI,ZENITH_ANGLE,EARTH_AZIMUTH_ANGLE,TIME,EVENT_ID,RUN_ID,RECON_VERSION,CALIB_VERSION [3],EVENT_CLASS [32],EVENT_TYPE [32],CONVERSION_TYPE,LIVETIME,DIFRSP0,DIFRSP1,DIFRSP2,DIFRSP3,DIFRSP4
MeV,deg,deg,deg,deg,deg,deg,deg,deg,s,,,,,,,,s,,,,,
float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,int32,int32,int16,int16,bool,bool,int16,float64,float32,float32,float32,float32,float32
145927.0,291.662,42.2341,74.5437,11.8678,38.0455,83.5358,55.6387,314.034,239561457.866,4851437,239559565,0,0 .. 0,False .. True,False .. True,0,275.698088974,2.69657e-11,3238.25,0.0,0.0,0.0
221273.0,46.9858,-40.6389,247.489,-58.8739,34.1051,224.209,68.2524,198.319,239562739.085,7521432,239559565,0,0 .. 0,False .. True,False .. True,0,64.7974931598,1.53573e-12,2774.2,0.0,0.0,0.0
57709.2,121.841,49.2288,169.868,32.3017,71.5636,34.2925,36.7173,25.5439,239563180.302,8690693,239559565,0,0 .. 0,False .. True,False .. True,0,30.57218647,8.11096e-12,253.221,0.0,0.0,0.0
221224.0,83.5626,-4.21744,207.783,-19.0771,20.5089,92.1605,32.3033,239.141,239563382.213,9208424,239559565,0,0 .. 0,False .. True,False .. True,0,27.4125095904,1.66075e-11,2980.12,0.0,0.0,0.0
698997.0,320.895,-1.32789,51.2218,-33.9718,35.3621,158.741,12.0867,72.2029,239566572.951,2480483,239565645,0,0 .. 0,False .. True,False .. True,0,106.475481123,2.26543e-13,2706.14,0.0,0.0,0.0


In [52]:
table.info('stats')           # <--  Table column info

<Table length=60978>
        name             mean           std            min           max     
------------------- -------------- -------------- ------------- -------------
             ENERGY       132805.0       166230.0       50000.2   1.99848e+06
                 RA        205.643        95.3355    0.00973502       359.994
                DEC       -4.83771        41.0807      -88.6767       89.3993
                  L        179.567        119.668    0.00330579       359.998
                  B        1.21523        27.4603      -89.7519        89.498
              THETA        40.1624        16.7082      0.109743       82.1941
                PHI        178.625        104.362    0.00225764       359.996
       ZENITH_ANGLE        56.4762        24.7502      0.286511       104.997
EARTH_AZIMUTH_ANGLE        178.775        98.4542    0.00501673       359.997
               TIME  341465163.041  61063749.8695 239561457.866 444433764.433
           EVENT_ID  5763331.11188  3700135

C:\Users\arjun_000\Anaconda3\lib\site-packages\astropy\table\column.py:263: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return self.data.__eq__(other)


In [53]:
cat = source_catalogs['2fhl']